In [1]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
%pip install "ultralytics<=8.3.40" supervision roboflow
# prevent ultralytics from tracking your activity
!yolo settings sync=False
import ultralytics
ultralytics.checks()

In [ ]:
!yolo task=detect mode=predict model=yolo11m-seg.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=True

In [ ]:
from IPython.display import Image as IPyImage

IPyImage(filename=f'{HOME}/runs/segment/predict/dog.jpg', width=600)

In [ ]:
from ultralytics import YOLO
from PIL import Image
import requests

model = YOLO('yolo11m-seg.pt')
image = Image.open(requests.get('https://media.roboflow.com/notebooks/examples/dog.jpeg', stream=True).raw)
result = model.predict(image, conf=0.25)[0]

In [ ]:
from roboflow import Roboflow

!roboflow workspace list

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

from google.colab import userdata
from roboflow import Roboflow

ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

workspace = rf.workspace("tbe") # Your workspace ID
project = workspace.project("rice-grain-svjri") # Your project ID
version = project.version(4)
dataset = version.download("yolov11")

In [ ]:
%cd {HOME}

!yolo task=segment mode=train model=yolo11m-seg.pt data={dataset.location}/data.yaml epochs=1000 imgsz=640 plots=True

In [ ]:
!ls {HOME}/runs/segment/train/

In [ ]:
# from IPython.display import Image as IPyImage

# IPyImage(filename=f'{HOME}/runs/segment/train/confusion_matrix.png', width=600)

In [ ]:
!yolo task=segment mode=val model={HOME}/runs/segment/train/weights/best.pt data={dataset.location}/data.yaml imgsz=640 plots=True max_det=1000

In [ ]:
# Data Augmentation

# Melakukan augmentasi pada dataset untuk meningkatkan variasi data training

In [ ]:
# Install library untuk augmentasi
%pip install albumentations opencv-python-headless

In [ ]:
import cv2
import numpy as np
import albumentations as A
from pathlib import Path
import shutil
from tqdm import tqdm

# Fungsi untuk membaca annotation YOLO format
def read_yolo_annotation(annotation_path):
    with open(annotation_path, 'r') as f:
        annotations = []
        for line in f.readlines():
            parts = line.strip().split()
            if len(parts) > 0:
                class_id = int(parts[0])
                coords = [float(x) for x in parts[1:]]
                annotations.append([class_id] + coords)
    return annotations

# Fungsi untuk menulis annotation YOLO format
def write_yolo_annotation(annotation_path, annotations):
    with open(annotation_path, 'w') as f:
        for ann in annotations:
            class_id = int(ann[0])
            coords = ' '.join([f'{x:.6f}' for x in ann[1:]])
            f.write(f'{class_id} {coords}\n')

# Fungsi untuk transformasi koordinat polygon
def transform_polygon_coords(coords, img_width, img_height, transform_matrix):
    """Transform polygon coordinates using transformation matrix"""
    transformed_coords = []
    for i in range(0, len(coords), 2):
        x = coords[i] * img_width
        y = coords[i + 1] * img_height

        # Apply transformation
        if transform_matrix == 'horizontal_flip':
            x = img_width - x
        elif transform_matrix == 'vertical_flip':
            y = img_height - y
        elif transform_matrix == 'rotate_90':
            x, y = img_height - y, x
        elif transform_matrix == 'rotate_180':
            x, y = img_width - x, img_height - y
        elif transform_matrix == 'rotate_270':
            x, y = y, img_width - x

        # Normalize back
        x_norm = x / img_width if transform_matrix not in ['rotate_90', 'rotate_270'] else x / img_height
        y_norm = y / img_height if transform_matrix not in ['rotate_90', 'rotate_270'] else y / img_width

        transformed_coords.extend([x_norm, y_norm])

    return transformed_coords

# Fungsi untuk apply transformasi individual dengan tracking
def apply_tracked_transform(image, annotations, img_width, img_height):
    """Apply transformation dan track transformasi untuk koordinat"""
    # Track transformasi yang diapply
    applied_transforms = []

    # Apply HorizontalFlip
    if np.random.random() < 0.5:
        image = cv2.flip(image, 1)
        applied_transforms.append('horizontal_flip')

    # Apply VerticalFlip
    if np.random.random() < 0.5:
        image = cv2.flip(image, 0)
        applied_transforms.append('vertical_flip')

    # Apply RandomRotate90
    rotate_choice = np.random.random()
    if rotate_choice < 0.5:
        k = np.random.choice([1, 2, 3])  # 90, 180, or 270 degrees
        image = np.rot90(image, k)
        if k == 1:
            applied_transforms.append('rotate_90')
        elif k == 2:
            applied_transforms.append('rotate_180')
        elif k == 3:
            applied_transforms.append('rotate_270')

    # Transform annotations
    transformed_annotations = []
    for ann in annotations:
        class_id = ann[0]
        coords = ann[1:]

        # Apply each transformation to coordinates
        current_coords = coords
        current_width = img_width
        current_height = img_height

        for transform_type in applied_transforms:
            current_coords = transform_polygon_coords(
                current_coords, current_width, current_height, transform_type
            )
            # Swap dimensions for rotation
            if transform_type in ['rotate_90', 'rotate_270']:
                current_width, current_height = current_height, current_width

        transformed_annotations.append([class_id] + current_coords)

    return image, transformed_annotations

print("Fungsi transformasi koordinat berhasil didefinisikan!")

In [ ]:
# Definisi pipeline augmentasi
# Transformasi geometri (flip, rotate) ditangani secara manual
# Albumentations hanya untuk transformasi pixel/color yang tidak ubah koordinat
import albumentations as A

transform_color = A.Compose([
    A.RandomBrightnessContrast(
        brightness_limit=0.2, 
        contrast_limit=0.2, 
        p=0.5
    ),
    A.HueSaturationValue(
        hue_shift_limit=10,
        sat_shift_limit=20,
        val_shift_limit=10,
        p=0.5
    ),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
    A.GaussianBlur(blur_limit=(3, 7), p=0.3),
    A.CLAHE(clip_limit=2.0, p=0.3),
], bbox_params=None)

print("Pipeline augmentasi berhasil didefinisikan!")
print("\nTeknik augmentasi yang digunakan:")
print("Transformasi Geometri (dengan transformasi koordinat):")
print("  1. HorizontalFlip (50%)")
print("  2. VerticalFlip (50%)")
print("  3. RandomRotate90 (50%)")
print("\nTransformasi Warna/Pixel:")
print("  4. RandomBrightnessContrast (50%)")
print("  5. HueSaturationValue (50%)")
print("  6. GaussNoise (30%)")
print("  7. GaussianBlur (30%)")
print("  8. CLAHE (30%)")

In [ ]:
# Visualisasi hasil augmentasi
import matplotlib.pyplot as plt
from PIL import Image
import random
import os
from pathlib import Path

def visualize_augmentations(images_path, num_samples=4):
    """Visualisasi gambar original dan hasil augmentasinya"""
    images_dir = Path(images_path)
    
    # Dapatkan gambar original (bukan hasil augmentasi)
    original_images = [f for f in os.listdir(images_path) if '_aug' not in f and f.endswith(('.jpg', '.jpeg', '.png'))]
    
    if len(original_images) == 0:
        print("Tidak ada gambar original ditemukan")
        return
    
    # Pilih beberapa gambar secara random
    sample_images = random.sample(original_images, min(num_samples, len(original_images)))
    
    for orig_name in sample_images:
        # Cari semua augmentasi dari gambar ini
        base_name = Path(orig_name).stem
        aug_images = [f for f in os.listdir(images_path) if f.startswith(base_name + '_aug')]
        
        # Siapkan plot
        num_cols = min(4, len(aug_images) + 1)
        fig, axes = plt.subplots(1, num_cols, figsize=(20, 5))
        
        if num_cols == 1:
            axes = [axes]
        
        # Tampilkan gambar original
        orig_img = Image.open(os.path.join(images_path, orig_name))
        axes[0].imshow(orig_img)
        axes[0].set_title('Original')
        axes[0].axis('off')
        
        # Tampilkan gambar hasil augmentasi
        for idx, aug_name in enumerate(aug_images[:num_cols-1]):
            aug_img = Image.open(os.path.join(images_path, aug_name))
            axes[idx+1].imshow(aug_img)
            axes[idx+1].set_title(f'Augmented {idx+1}')
            axes[idx+1].axis('off')
        
        plt.tight_layout()
        plt.show()
        print(f"Gambar: {orig_name}")
        print("-" * 50)

# NOTE: the actual call to visualize_augmentations is moved later to after augmentation run/verification

In [ ]:
# Konfigurasi path
DATASET_PATH = "/content/datasets/Rice-Grain-4"
TRAIN_IMAGES_PATH = f"{DATASET_PATH}/train/images"
TRAIN_LABELS_PATH = f"{DATASET_PATH}/train/labels"

# Jumlah augmentasi per gambar
NUM_AUGMENTATIONS = 3  # Setiap gambar akan diaugmentasi 3 kali

print(f"Dataset path: {DATASET_PATH}")
print(f"Train images path: {TRAIN_IMAGES_PATH}")
print(f"Train labels path: {TRAIN_LABELS_PATH}")
print(f"Jumlah augmentasi per gambar: {NUM_AUGMENTATIONS}")

In [ ]:
# Fungsi untuk melakukan augmentasi pada segmentation dataset
from pathlib import Path

def augment_segmentation_dataset(images_path, labels_path, num_augmentations=3):
    """
    Augmentasi dataset dengan format YOLO segmentation
    """
    images_dir = Path(images_path)
    labels_dir = Path(labels_path)
    
    # Dapatkan semua file gambar
    image_files = list(images_dir.glob('*.jpg')) + list(images_dir.glob('*.jpeg')) + list(images_dir.glob('*.png'))
    
    print(f"Ditemukan {len(image_files)} gambar untuk diaugmentasi")
    
    augmented_count = 0
    
    for img_path in tqdm(image_files, desc="Augmentasi gambar"):
        # Baca gambar
        image = cv2.imread(str(img_path))
        if image is None:
            print(f"Gagal membaca gambar: {img_path}")
            continue
            
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_height, img_width = image.shape[:2]
        
        # Path annotation
        label_path = labels_dir / f"{img_path.stem}.txt"
        
        # Baca annotations jika ada
        if not label_path.exists():
            print(f"Tidak ada label untuk: {img_path.name}")
            continue
        
        annotations = read_yolo_annotation(label_path)
        
        # Lakukan augmentasi sebanyak num_augmentations
        for aug_idx in range(num_augmentations):
            # Step 1: Apply transformasi geometri (flip, rotate) dengan tracking koordinat
            augmented_image, transformed_annotations = apply_tracked_transform(
                image.copy(), annotations, img_width, img_height
            )
            
            # Step 2: Apply transformasi warna/pixel (tidak ubah koordinat)
            color_transformed = transform_color(image=augmented_image)
            augmented_image = color_transformed['image']
            
            # Simpan gambar hasil augmentasi
            aug_img_name = f"{img_path.stem}_aug{aug_idx}{img_path.suffix}"
            aug_img_path = images_dir / aug_img_name
            
            augmented_image_bgr = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)
            cv2.imwrite(str(aug_img_path), augmented_image_bgr)
            
            # Simpan annotation yang sudah ditransformasi
            aug_label_name = f"{img_path.stem}_aug{aug_idx}.txt"
            aug_label_path = labels_dir / aug_label_name
            write_yolo_annotation(aug_label_path, transformed_annotations)
            
            augmented_count += 1
    
    print(f"\nSelesai! Total {augmented_count} gambar baru telah dibuat")
    print(f"Dataset sekarang memiliki {len(image_files) + augmented_count} gambar training")
    
    return augmented_count

print("Fungsi augment_segmentation_dataset berhasil didefinisikan!")

In [ ]:
# Jalankan augmentasi
print("Memulai proses augmentasi...")
print("=" * 50)

augmented_count = augment_segmentation_dataset(
    images_path=TRAIN_IMAGES_PATH,
    labels_path=TRAIN_LABELS_PATH,
    num_augmentations=NUM_AUGMENTATIONS,
)

print("=" * 50)
print("Augmentasi selesai!")
print(f"Total gambar baru: {augmented_count}")

In [ ]:
# Verifikasi hasil augmentasi
import os

train_images = len(os.listdir(TRAIN_IMAGES_PATH))
train_labels = len([f for f in os.listdir(TRAIN_LABELS_PATH) if f.endswith('.txt')])

print("Statistik Dataset Setelah Augmentasi:")
print("=" * 50)
print(f"Jumlah gambar training: {train_images}")
print(f"Jumlah label training: {train_labels}")
print(f"Path dataset: {DATASET_PATH}")

# Tampilkan beberapa nama file hasil augmentasi
print("\nContoh file hasil augmentasi:")
aug_files = [f for f in os.listdir(TRAIN_IMAGES_PATH) if '_aug' in f]
for i, f in enumerate(aug_files[:5]):
    print(f"  {i+1}. {f}")

In [ ]:
# Tampilkan visualisasi hasil augmentasi
visualize_augmentations(TRAIN_IMAGES_PATH, num_samples=2)